In [1]:
import pandas as pd
import requests
import re
import json
from bs4 import BeautifulSoup

In [2]:
url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901'
# https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901
# 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410020101'
# https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410037101
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
result = find_between(soup.prettify(), 'tableContainerElement = $(".tableContainer").clone();', 'window.addEventListener("resize", function() {') + 'end'
data = find_between(result, 'prepareTable(', '\n')[:-2]

In [5]:
json_data = json.loads(data)

In [6]:
rows = json_data['rows']
new_rows = []
for row in rows:
    values = row['values']
    for value in values:
        new_rows.append(value["value"])

In [7]:
headers = json_data['headers']["columnHeaders"]
header_values = []
for header in headers:
    if header["name"] == "Reference period":
        for item in header["values"]:
            header_values.append(item["value"])
print(header_values)


['June 2022', 'July 2022', 'August 2022', 'September 2022', 'October 2022']


In [8]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [9]:
rows_values = {}
key = ""
data = []
index = 0
for row in new_rows:
    temp_data = {}
    if not isfloat(row):
        key = row
        rows_values[key] = []
        data = []
        index = 0
    if isfloat(row):
        data.append(float(row))
        rows_values[key] = data
        index += 1


In [10]:
test_data = []
for key, value in rows_values.items():
    index=0
    temp_data = {}
    temp_data["key"] = key
    for i in value:
        temp_data[header_values[index]] = i
        index += 1
    test_data.append(temp_data)

In [12]:
df = pd.DataFrame(test_data)

In [13]:
print(df.head())

                                                 key   June 2022   July 2022  \
0  Industrial aggregate including unclassified bu...  17896465.0  17667499.0   
1  Industrial aggregate excluding unclassified bu...  17527724.0  17309064.0   
2                         Goods producing industries   3142349.0   3158518.0   
3                      Forestry, logging and support     41935.0     43924.0   
4      Mining, quarrying, and oil and gas extraction    210019.0    212484.0   

   August 2022  September 2022  October 2022  
0   17423222.0      17891150.0    17890055.0  
1   17067748.0      17532721.0    17542835.0  
2    3140232.0       3165074.0     3129926.0  
3      42685.0         40752.0       39170.0  
4     211469.0        212403.0      210817.0  
